In [1]:
import joblib
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict , load_from_disk
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pd.set_option('display.max_colwidth', None) 

/home/d3v/anaconda3/envs/wysa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/d3v/anaconda3/envs/wysa/lib/python3.9/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
class MultitaskModel(nn.Module):
    def __init__(self, model_name, num_product_labels, num_emotion_labels):
        super(MultitaskModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        
        self.product_classifier = nn.Linear(hidden_size, num_product_labels)
        self.emotion_classifier = nn.Linear(hidden_size, num_emotion_labels)
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,  # labels is a tensor of shape (batch_size, 2)
    ):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] token representation
        pooled_output = self.dropout(pooled_output)
        
        product_logits = self.product_classifier(pooled_output)
        emotion_logits = self.emotion_classifier(pooled_output)
        
        loss = None
        if labels is not None:
            product_labels = labels[:, 0]
            emotion_labels = labels[:, 1]
            loss_fct = nn.CrossEntropyLoss()
            product_loss = loss_fct(product_logits, product_labels)
            emotion_loss = loss_fct(emotion_logits, emotion_labels)
            loss = product_loss + emotion_loss  # Combine losses
        
        return {
            'loss': loss,
            'logits': (product_logits, emotion_logits)
        }

In [3]:
product_encoder = joblib.load('product_encoder.joblib')
emotion_encoder = joblib.load('emotion_encoder.joblib')
num_product_labels, num_emotion_labels = len(product_encoder.classes_) , len(emotion_encoder.classes_)

In [5]:
model = MultitaskModel(
    model_name='bert-base-uncased', 
    num_product_labels=num_product_labels,
    num_emotion_labels=num_emotion_labels,
)

In [6]:
state_dict = torch.load('MODEL_04/pytorch_model.bin', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()

/tmp/ipykernel_6211/2583355429.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('MODEL_04/pytorch_model.bin', map_location=torch.device('cpu'))


MultitaskModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [13]:
tokenizer = AutoTokenizer.from_pretrained('MODEL_04/')

def predict_text(texts):
    emotion_predictions = []
    product_predictions = []

    for text in texts:
        # Tokenize the raw text
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to model device

        # Remove token_type_ids if not required by the model
        if 'token_type_ids' in inputs and 'token_type_ids' not in model.forward.__code__.co_varnames:
            del inputs['token_type_ids']

        with torch.no_grad():
            outputs = model(**inputs)

            if isinstance(outputs['logits'], tuple):
                product_logits = outputs['logits'][0]
                emotion_logits = outputs['logits'][1]
            else:
                # If it's a single tensor, slice it based on the number of classes
                product_logits = outputs['logits'][:, len(product_encoder.classes_):]
                emotion_logits = outputs['logits'][:, :len(emotion_encoder.classes_)]

            emotion_pred_index = torch.argmax(emotion_logits, dim=1).item()
            product_pred_index = torch.argmax(product_logits, dim=1).item()

            # Map indices to class names
            emotion_pred_class = emotion_encoder.inverse_transform([emotion_pred_index])[0]
            product_pred_class = product_encoder.inverse_transform([product_pred_index])[0]

        emotion_predictions.append(emotion_pred_class)
        product_predictions.append(product_pred_class)

    return [{"emotion_prediction": e, "product_prediction": p} for e, p in zip(emotion_predictions, product_predictions)]


In [15]:
raw_texts = ["Sample text 1", "I just love the way IOS animate the new message app"] 
predictions = predict_text(raw_texts)
print(predictions)


[{'emotion_prediction': 'No emotion toward brand or product', 'product_prediction': 'No Product'}, {'emotion_prediction': 'Positive emotion', 'product_prediction': 'iPad or iPhone App'}]
